In [2]:
"""
NOTES:
- for n=2, a Cholesky decomposition exists but for n >= 3 a brute force search of lower triangular matrices over GF(3^2) for one of the 3x3 matrices U associated to an S_n-invariant symmetric bilinear form reveals there is no such matrix L such that U = LL^*
- this is because the lower triangular condition is too stringent. there is a decomposition if we allow general matrices, which can be computed using BaseChangeToCanonical in the GAP `forms` package.

""";

In [5]:
U_0 = matrix(GF(3^2),[[1,2],[2,1]]); U_0

[1 2]
[2 1]

In [20]:
U_1 = matrix(GF(3^2),[[1,2,2],[2,1,2],[2,2,1]]); U_1

[1 2 2]
[2 1 2]
[2 2 1]

In [21]:
U_2 = matrix(GF(3^2),[[0,1,2],[1,0,1],[2,1,0]]); U_2

[0 1 2]
[1 0 1]
[2 1 0]

In [4]:
"""
gap> LoadPackage("forms");
forms has BaseChangeToCanonical, which according to the docs, does the base change you'd like to see.
E.g.

$ gap
 ┌───────┐   GAP 4.13.1 of 2024-06-11
 │  GAP  │   https://www.gap-system.org
 └───────┘   Architecture: amd64
 Configuration:  gmp 6.3.0, GASMAN, readline
 Loading the library and packages ...
 Packages:   Alnuth 3.2.1, AtlasRep 2.1.8, AutPGrp 1.11, CRISP 1.4.6, CTblLib 1.3.9, 
             FactInt 1.6.3, FGA 1.5.0, GAPDoc 1.6.7, IO 4.8.2, IRREDSOL 1.4.4, LAGUNA 3.9.6, 
             Polenta 1.3.10, Polycyclic 2.16, PrimGrp 3.4.4, RadiRoot 2.9, ResClasses 4.7.3, 
             SmallGrp 1.5.3, Sophus 1.27, TomLib 1.2.11, TransGrp 3.6.5, utils 0.84
 Try '??help' for help. See also '?copyright', '?cite' and '?authors'
gap> LoadPackage("forms");
---------------------------------------------------------------------
Loading 'Forms' 1.2.12 (30/08/2024)
by John Bamberg (http://school.maths.uwa.edu.au/~bamberg/)
   Jan De Beule (http://www.debeule.eu)
For help, type: ?Forms 
---------------------------------------------------------------------
true
gap> u2:=Z(3)^0*[[0,1,2],[1,0,1],[2,1,0]];
[ [ 0*Z(3), Z(3)^0, Z(3) ], [ Z(3)^0, 0*Z(3), Z(3)^0 ], [ Z(3), Z(3)^0, 0*Z(3) ] ]
gap> form := HermitianFormByMatrix( u2, GF(3^2) );
< hermitian form >
gap> b:=BaseChangeToCanonical(form);
[ [ Z(3)^0, Z(3^2), 0*Z(3) ], [ Z(3^2)^3, Z(3^2)^2, 0*Z(3) ], [ Z(3^2)^3, Z(3^2)^7, Z(3^2)^7 ] 
 ]
gap> bstar:=List(TransposedMat(b),x->List(x,y->y^3));
[ [ Z(3)^0, Z(3^2), Z(3^2) ], [ Z(3^2)^3, Z(3^2)^6, Z(3^2)^5 ], [ 0*Z(3), 0*Z(3), Z(3^2)^5 ] ]
gap> Display(b*u2*bstar);
 1 . .
 . 1 .
 . . 1
gap>
""";

In [34]:
"""
sage: gap_console();
 ┌───────┐   GAP 4.12.2 of 2022-12-18
 │  GAP  │   https://www.gap-system.org
 └───────┘   Architecture: aarch64-apple-darwin23-default64-kv8
 Configuration:  gmp 6.3.0, GASMAN, readline
 Loading the library and packages ...
 Packages:   GAPDoc 1.6.6, PrimGrp 3.4.3, SmallGrp 1.5.1, TransGrp 3.6.3
 Try '??help' for help. See also '?copyright', '?cite' and '?authors'
gap> LoadPackage("forms");
---------------------------------------------------------------------
Loading 'Forms' 1.2.12 (30/08/2024)
by John Bamberg (http://school.maths.uwa.edu.au/~bamberg/)
   Jan De Beule (http://www.debeule.eu)
For help, type: ?Forms 
---------------------------------------------------------------------
true
gap> u1 := Z(3)^0*[[1,2,2],[2,1,2],[2,2,1]];
[ [ Z(3)^0, Z(3), Z(3) ], [ Z(3), Z(3)^0, Z(3) ], [ Z(3), Z(3), Z(3)^0 ] ]
gap> form := HermitianFormByMatrix(u1, GF(3^2));
< hermitian form >
gap> b:=BaseChangeToCanonical(form);
[ [ Z(3)^0, 0*Z(3), 0*Z(3) ], [ Z(3^2)^2, Z(3)^0, Z(3^2) ], [ Z(3), Z(3^2)^3, Z(3^2)^2 ] ]
gap> bstar:=List(TransposedMat(b),x->List(x,y->y^3));
[ [ Z(3)^0, Z(3^2)^6, Z(3) ], [ 0*Z(3), Z(3)^0, Z(3^2) ], [ 0*Z(3), Z(3^2)^3, Z(3^2)^6 ] ]
gap> (b^-1)*(bstar^-1);
[ [ Z(3)^0, Z(3), Z(3) ], [ Z(3), Z(3)^0, Z(3) ], [ Z(3), Z(3), Z(3)^0 ] ]
gap> (b^-1)*(bstar^-1) = u1;
true
gap> b^-1;
[ [ Z(3)^0, 0*Z(3), 0*Z(3) ], [ Z(3), Z(3^2)^3, Z(3^2)^6 ], [ Z(3), Z(3)^0, Z(3^2) ] ]
""";

In [50]:
#define conjugation as x |--> x**q, an order two automorphism of F_q^2. note x**q == x for x \in F_q.
def conjugate_pos_char(A):
    assert A.nrows() == A.ncols()
    field_size = A.base_ring().order()
    q = sqrt(field_size) if field_size.is_square() else field_size
    return matrix(A.base_ring(),[[A[i][j]**q for j in range(A.nrows())] for i in range(A.nrows())])

In [100]:
#the GAP version of the base change matrix b associate to u1
#b := [[Z(3)^0, 0*Z(3), 0*Z(3)],[Z(3^2)^2,Z(3)^0, Z(3^2)], [Z(3),Z(3^2)^3,Z(3^2)^2]]

F = GF(3^2, name='z', modulus="primitive") #ISSUE: need to ensure GAP generator Z(3^2) matches Sage generator z = F.gen()

# Define the matrix using the elements from GAP
b = Matrix(F, [
    [1, 0, 0],           # First row: Z(3)^0, 0*Z(3), 0*Z(3)
    [z^2, 1, z],         # Second row: Z(3^2)^2, Z(3)^0, Z(3^2)
    [2, z^3, z^2]        # Third row: Z(3), Z(3^2)^3, Z(3^2)^2
])

bstar = conjugate_pos_char(b).transpose()
b.inverse()*bstar.inverse()

[1 2 2]
[2 1 2]
[2 2 1]

In [28]:
U_1

[1 2 2]
[2 1 2]
[2 2 1]